In [1]:
import tensorflow as tf
import numpy as np
from datetime import datetime

learning_rate = 0.01
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
data = np.loadtxt('C:/Users/Sunwoo/workspace/winequality-red2.csv', delimiter=';')
print(data)

[[ 7.4    0.7    0.    ...  0.56   9.4    5.   ]
 [ 7.8    0.88   0.    ...  0.68   9.8    5.   ]
 [ 7.8    0.76   0.04  ...  0.65   9.8    5.   ]
 ...
 [ 6.3    0.51   0.13  ...  0.75  11.     6.   ]
 [ 5.9    0.645  0.12  ...  0.71  10.2    5.   ]
 [ 6.     0.31   0.47  ...  0.66  11.     6.   ]]


In [3]:
x_data = data[:, 0:-1]    #fixed acidity;"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol"
y_data = data[:, [-1]]    #"quality"
print(x_data)
print(y_data)

[[ 7.4    0.7    0.    ...  3.51   0.56   9.4  ]
 [ 7.8    0.88   0.    ...  3.2    0.68   9.8  ]
 [ 7.8    0.76   0.04  ...  3.26   0.65   9.8  ]
 ...
 [ 6.3    0.51   0.13  ...  3.42   0.75  11.   ]
 [ 5.9    0.645  0.12  ...  3.57   0.71  10.2  ]
 [ 6.     0.31   0.47  ...  3.39   0.66  11.   ]]
[[5.]
 [5.]
 [5.]
 ...
 [6.]
 [5.]
 [6.]]


In [4]:
#1 if y >= 7, 0 if y < 7
y_data = np.array([1 if i>=7 else 0 for i in y_data]).reshape(len(x_data), 1)
print(y_data)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [5]:
w1 = tf.Variable(tf.random.uniform([11, 20], dtype=tf.float64))
b1 = tf.Variable(tf.random.uniform([20], dtype=tf.float64))
w2 = tf.Variable(tf.random.uniform([20, 20], dtype=tf.float64))
b2 = tf.Variable(tf.random.uniform([20], dtype=tf.float64))
w3 = tf.Variable(tf.random.uniform([20, 20], dtype=tf.float64))
b3 = tf.Variable(tf.random.uniform([20], dtype=tf.float64))
w4 = tf.Variable(tf.random.uniform([20, 1], dtype=tf.float64))
b4 = tf.Variable(tf.random.uniform([1], dtype=tf.float64))

def hypothesis() :
    layer1 = tf.sigmoid(tf.matmul(x_data, w1) + b1)
    layer2 = tf.sigmoid(tf.matmul(layer1, w2) + b2)
    layer3 = tf.sigmoid(tf.matmul(layer2, w3) + b3)
    h = tf.sigmoid(tf.matmul(layer3, w4) + b4)
    return h

@tf.function
def costFunc() :
    return -tf.reduce_mean(y_data * tf.math.log(hypothesis()) + (1 - y_data) * tf.math.log(1 - hypothesis()))

stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
logdir = 'logs/mylogs/%s' % stamp
writer = tf.summary.create_file_writer(logdir)
tf.summary.trace_on(graph=True, profiler=True)
costFunc()
with writer.as_default() :
    tf.summary.trace_export(name='graph_t1', step=0, profiler_outdir=logdir)

opt = tf.keras.optimizers.SGD(learning_rate)
for step in range(10001) :
    opt.minimize(costFunc, var_list=[w1, b1, w2, b2, w3, b3, w4, b4])
    predicted = tf.cast(hypothesis() > 0.5, dtype=tf.float64)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, y_data), dtype=tf.float64))
    if step % 1000 == 0 :
        print("Epoch : %.f, loss : %.4f" % (step, costFunc()))
print("Hypothesis :", hypothesis().numpy(), "Predicted :", predicted.numpy(), "Accuracy :", accuracy.numpy())

Instructions for updating:
use `tf.profiler.experimental.start` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Epoch : 0, loss : 9.3157
Epoch : 1000, loss : 0.3971
Epoch : 2000, loss : 0.3971
Epoch : 3000, loss : 0.3971
Epoch : 4000, loss : 0.3971
Epoch : 5000, loss : 0.3971
Epoch : 6000, loss : 0.3971
Epoch : 7000, loss : 0.3971
Epoch : 8000, loss : 0.3971
Epoch : 9000, loss : 0.3971
Epoch : 10000, loss : 0.3971
Hypothesis : [[0.13570982]
 [0.13570982]
 [0.13570982]
 ...
 [0.13570982]
 [0.13570982]
 [0.13570982]] Predicted : [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]] Accuracy : 0.8642901813633521


In [6]:
%load_ext tensorboard
%tensorboard --logdir=logs/mylogs

Reusing TensorBoard on port 6006 (pid 8428), started 7 days, 5:42:08 ago. (Use '!kill 8428' to kill it.)